# SIMULAÇÃO DE VISUALIZAÇÃO DO USUÁRIO

In [1]:
company = Company.objects.filter(name__iexact="GES - UHE-CANA BRAVA").first()
company

<Company: cff59760-69dd-4d07-bba5-61aa8b6faeea: GES - UHE-CANA BRAVA>

In [13]:
user = User.objects.get(username="tulio.ribeiral")
user

<User: tulio.ribeiral - Tulio Ribeiral>

### EQUIPES DO USUÁRIO

In [14]:
user_firms = list((user.user_firms.all()).union(
                    user.user_firms_manager.all()))
            
user_firms

[<Firm: [2fd0ca76-b741-47d7-af9c-607e6ed1eca2: GES - UHE-MIRANDA] c74bba36-3dd0-406a-8f67-eb9f2b3d2f18: Engie Brasil Serbiços>,
 <Firm: [296edae1-b154-40ff-87b1-3b78e3441906: GES - UHE-PONTE DE PEDRA] c9a95793-b091-45c8-81d7-2510487218aa: Homologadores>,
 <Firm: [296edae1-b154-40ff-87b1-3b78e3441906: GES - UHE-PONTE DE PEDRA] d25df098-16d5-4c8f-99b7-3de0c373b3f3: MEIO AMBIENTE>,
 <Firm: [17663a30-5f71-4e7c-8ac2-adf472342b42: GES - UHE-JAGUARA] 05e3f1c6-0353-4909-b6bd-a193b4caa2a7: MEIO AMBIENTE>,
 <Firm: [2fd0ca76-b741-47d7-af9c-607e6ed1eca2: GES - UHE-MIRANDA] 89d13730-1ea7-4531-979d-9a182c673ef5: Seleta Ambiental>,
 <Firm: [2fd0ca76-b741-47d7-af9c-607e6ed1eca2: GES - UHE-MIRANDA] 4a39a9ac-c966-42ee-b07a-27414dffbe50: Raiz Ambiental PEA>,
 <Firm: [2fd0ca76-b741-47d7-af9c-607e6ed1eca2: GES - UHE-MIRANDA] adb600ba-0773-4174-a4dc-5b3f9488d5a9: MEIO AMBIENTE>,
 <Firm: [2fd0ca76-b741-47d7-af9c-607e6ed1eca2: GES - UHE-MIRANDA] b32e0631-e951-45f7-b19c-3d108e87a697: Saúde e Segurança>,
 <Firm

In [17]:
[f'Usina:{a.company.name} | Equipe:{a.name}' for a in user_firms if a.can_use_ecm_integration == True]

['Usina:GES - UHE-PONTE DE PEDRA | Equipe:MEIO AMBIENTE',
 'Usina:GES - UHE-JAGUARA | Equipe:MEIO AMBIENTE',
 'Usina:GES - UHE-MIRANDA | Equipe:MEIO AMBIENTE',
 'Usina:GES - UHE-JAGUARA | Equipe:Homologadores']

In [12]:
print('user: engie'),[(a.company.name,a.name) for a in user_firms if a.can_use_ecm_integration == True]

user: engie


(None,
 [('GES - UHE-MACHADINHO', 'MEIO AMBIENTE'),
  ('GES - UHE-SÃO SALVADOR', 'Homologadores'),
  ('GES - UHE-MIRANDA', 'Suporte'),
  ('GES - UHE-PASSO FUNDO', 'MEIO AMBIENTE'),
  ('GES - UHE-ITA', 'MEIO AMBIENTE')])

In [ ]:
Usina:GES - UHE-PONTE DE PEDRA | Equipe:MEIO AMBIENTE
Usina:GES - UHE-JAGUARA | Equipe:MEIO AMBIENTE
Usina:GES - UHE-MIRANDA | Equipe:MEIO AMBIENTE
Usina:GES - UHE-JAGUARA | Equipe:Homologadores

### VISUALIZAÇÃO DAS PROGRAMAÇÕES QUE O USUÁRIO PODE VER
##### POR COMPANY, 
##### ONDE O USUÁRIO POSSA SER RESPONSÁVEL PELA PROGRAMAÇÃO OU
##### CRIADOR DA PROGRAMAÇÃO OU
##### VISUALIZADOR DA PROGRAMAÇÃO OU
##### FAZ PARTE DA EQUIPE DA PROGRAMAÇÃO OU
##### FAZ PARTE DA EQUIPE VISUALIZADORA DA PROGRAMAÇÃO

In [9]:
jobs_all = Job.objects.filter(
                Q(company_id=company)
                & (
                    Q(worker=user)
                    | Q(created_by=user)
                    | Q(watcher_users=user)
                    | Q(firm__in=user_firms)
                    | Q(watcher_firms__in=user_firms)
                )
            ).distinct()
jobs_open = Job.objects.filter(
                Q(company_id=company)
                & (
                    Q(worker=user)
                    | Q(created_by=user)
                    | Q(watcher_users=user)
                    | Q(firm__in=user_firms)
                    | Q(watcher_firms__in=user_firms)
                ),
                archived='False'
            ).distinct().order_by('-start_date')
jobs_all.count(),jobs_open.count(),jobs_open[0].__dict__

(2221,
 764,
 {'_state': <django.db.models.base.ModelState at 0x7f51379d7e80>,
  'uuid': UUID('167daa93-128a-4017-9975-c3f5d3d7c259'),
  'company_id': UUID('31938021-6b32-461b-821c-4759a5016441'),
  'number': 'AFD-job-2024.1113',
  'title': '05/03 - Multifuncional Lote 01',
  'description': '',
  'start_date': datetime.datetime(2024, 3, 5, 10, 0, tzinfo=<UTC>),
  'end_date': None,
  'progress': 0.0,
  'executed_reportings': 0,
  'reporting_count': 2,
  'worker_id': UUID('910d785b-d4f6-4b8b-8d6d-421fe762ddcd'),
  'firm_id': UUID('47f332db-6b9e-4e2a-b745-ff5fdac561ca'),
  'inspection_id': None,
  'parent_inventory_id': None,
  'metadata': {},
  'created_by_id': UUID('f4204ee0-2084-4bff-b3b6-ecee2ce40808'),
  'archived': False,
  'creating_batches': False,
  'pending_inventory_to_reporting_id': None,
  'total_inventory_batches': 0,
  'total_reporting_in_reporting_batches': 0,
  'last_notification_sent_at': None})

In [12]:
#Primeiras 15 programações
for i,a in enumerate(jobs_open):
    if '2023.3732' in a.number:
        print(i,a.number)

166 AFD-job-2023.3732


In [9]:
values=[]
for a in jobs_open:
    if a.reporting_count>=100 and a.archived == False:
        obj={}
        obj['serial'] = a.number
        obj['apontamentos_contagem'] = a.reporting_count
#         obj['job'] = a
        values.append(obj)
    else:
        pass
#         print('Não há programações com + 50 apontamentos')
newlist = sorted(values, key=lambda d: d['apontamentos_contagem'],reverse=True)
len(newlist),newlist

(30,
 [{'serial': 'ENT-job-2024.77', 'apontamentos_contagem': 14685},
  {'serial': 'ENT-job-2023.1150', 'apontamentos_contagem': 390},
  {'serial': 'ENT-job-2023.1151', 'apontamentos_contagem': 387},
  {'serial': 'ENT-job-2023.1295', 'apontamentos_contagem': 351},
  {'serial': 'ENT-job-2023.1147', 'apontamentos_contagem': 331},
  {'serial': 'ENT-job-2023.934', 'apontamentos_contagem': 266},
  {'serial': 'ENT-job-2023.1149', 'apontamentos_contagem': 254},
  {'serial': 'ENT-job-2023.131', 'apontamentos_contagem': 185},
  {'serial': 'ENT-job-2023.1514', 'apontamentos_contagem': 166},
  {'serial': 'ENT-job-2023.1521', 'apontamentos_contagem': 166},
  {'serial': 'ENT-job-2024.7', 'apontamentos_contagem': 154},
  {'serial': 'ENT-job-2023.1163', 'apontamentos_contagem': 146},
  {'serial': 'ENT-job-2021.1559', 'apontamentos_contagem': 146},
  {'serial': 'ENT-job-2023.1148', 'apontamentos_contagem': 140},
  {'serial': 'ENT-job-2023.1076', 'apontamentos_contagem': 134},
  {'serial': 'ENT-job-202

In [15]:
rdos = MultipleDailyReport.objects.filter(
                Q(company_id=company)
                & (
                    Q(firm__in=user_firms)
                )
            ).distinct().order_by('-created_at')
rdos.count()

3396

In [16]:
rdos[:7],rdos[0].

<QuerySet [<MultipleDailyReport: [Arteris Litoral Sul - Conserva] 5bfb5bcd-749c-41be-859b-4fba4a4e534b - 2023-10-06>, <MultipleDailyReport: [Arteris Litoral Sul - Conserva] 6a59678e-0730-4cb8-b29a-92a1cd7dec90 - 2023-10-05>, <MultipleDailyReport: [Arteris Litoral Sul - Conserva] e91614ae-1136-4e90-8f82-4ce99bad4334 - 2023-10-04>, <MultipleDailyReport: [Arteris Litoral Sul - Conserva] 9d35c580-78f0-44c5-b889-a186d4aed994 - 2023-10-06>, <MultipleDailyReport: [Arteris Litoral Sul - Conserva] 5de0d762-ce97-4152-9b4f-ed8c0ba8fdb0 - 2023-10-05>, <MultipleDailyReport: [Arteris Litoral Sul - Conserva] 5fbaa946-bc79-4969-ad85-4d59b0bd085b - 2023-10-04>, <MultipleDailyReport: [Arteris Litoral Sul - Conserva] 415d1ebf-13cc-4010-82d9-f1452f0698d7 - 2023-10-06>]>

In [21]:
for a in DailyReportEquipment.objects.filter(multiple_daily_reports__in=list(rdos[:7])):
    print(a.description,a.kind,a.amount)

None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0
None None 1.0


In [27]:
DailyReportEquipment.objects.filter(multiple_daily_reports__in=list(rdos[:7]))[8].__dict__

{'_state': <django.db.models.base.ModelState at 0x7fd0586789d0>,
 'uuid': UUID('ea94646d-1b2b-4142-a217-12686d6f57a7'),
 'created_by_id': None,
 'company_id': UUID('52bf5f0c-7d75-4081-8616-06e0b43dee29'),
 'kind': None,
 'description': None,
 'amount': 1.0,
 'creation_date': datetime.datetime(2023, 8, 17, 6, 35, 30, 269602, tzinfo=<UTC>),
 'total_price': 0.0,
 'contract_item_administration_id': UUID('e2f072cf-e0ef-4418-9d6f-67211d3198fa'),
 'approval_status': 'WAITING_APPROVAL',
 'approval_date': None,
 'approved_by_id': None,
 'measurement_bulletin_id': None,
 '_fieldsignals_originals': {(140533378289488,
   140533147281024): {'uuid': UUID('ea94646d-1b2b-4142-a217-12686d6f57a7'), 'company': UUID('52bf5f0c-7d75-4081-8616-06e0b43dee29'), 'amount': 1.0, 'creation_date': datetime.datetime(2023, 8, 17, 6, 35, 30, 269602, tzinfo=<UTC>), 'total_price': 0.0, 'contract_item_administration': UUID('e2f072cf-e0ef-4418-9d6f-67211d3198fa'), 'approval_status': 'WAITING_APPROVAL'}}}

In [6]:
Job._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.CharField: number>,
 <django.db.models.fields.CharField: title>,
 <django.db.models.fields.CharField: description>,
 <django.db.models.fields.DateTimeField: start_date>,
 <django.db.models.fields.DateTimeField: end_date>,
 <django.db.models.fields.FloatField: progress>,
 <django.db.models.fields.IntegerField: executed_reportings>,
 <django.db.models.fields.IntegerField: reporting_count>,
 <django.db.models.fields.related.ForeignKey: worker>,
 <django.db.models.fields.related.ForeignKey: firm>,
 <django.db.models.fields.related.ForeignKey: inspection>,
 <django.db.models.fields.related.ForeignKey: parent_inventory>,
 <django.contrib.postgres.fields.jsonb.JSONField: metadata>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.BooleanField: archived>,
 <django.db.models.fields.BooleanField: creating_batches>,
 <django.contrib.post

In [16]:
user_firms[0].users.all()

<QuerySet [<User: eduarda.beraldo - Eduarda Beraldo>, <User: marcoantonio.reis - Marco Antonio Reis>, <User: gessica.gomes.ter - Gessica Gomes>, <User: saimon.sassaki - Saimon Sassaki>, <User: saullo.mendes - Saullo Mendes>, <User: rodolfo.rodrigues - Rodolfo Rodrigues>, <User: luan.poletti - Luan Poletti>]>

In [15]:
for a in user_firms:
    a.users.add(*User.objects.filter(username='luan.poletti'))